In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
# Load dataset
df = pd.read_csv('../Datasets/CleanData.csv')

In [ ]:
# Dataset info
df.shape
df.columns
df.dtypes
df.isnull().sum()

In [ ]:
# Extract features and target
X = df.iloc[:, 0:7].values
Y = df.iloc[:, 7].values

In [ ]:
# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [ ]:
# Define KNN model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [ ]:
# Evaluate KNN
from sklearn.metrics import accuracy_score, classification_report
knn_acc = accuracy_score(y_test, y_pred)
knn_cls = classification_report(y_test, y_pred)
knn_acc
knn_cls